## CMPT 2021: Algorithms
**Assignment:** Design of a Dynamic Ticketing System

**Student:** Paula Frossard Moreira

This project aims to present a Dynamic Ticketing System that
efficiently handles event ticket bookings, cancellations, and modifications while managing
overbooked requests using a waitlist system.

The solution should show the implementation of
data structures such as **dynamic arrays**, **hash tables**, and **priority queues** to achieve these
functionalities.

# Scenario

"Imagine a popular band, is performing at a 500-seat concert venue in Edmonton. The event is in
high demand, and ticket booking is managed through a Dynamic Ticketing System to ensure
fairness, efficiency, and priority-based handling of overbooking situations. Now develop a python
program with following features. "

# A. Ticket Booking

*   The program will show all 500 seats in a 20X25 grid at the beginning. Then a customer will be able to select a seat with priority and booking related information
(such as name, mobile number, email address).

*   After booking, the program will assign a unique Ticket ID/Reference for each
booking.
*   If no seats are available, your program will add the customer to a waitlist managed by
a priority queue.

Understanding logic of tasks

    #first, we need to initialize the system
    init 20x25 grid (representing 500 seats)
    
    #showing seats options
    function show_seats:
      for each row in grid:
        for each seat in row:
          if seat is available:
            show seat number
          else:
            show seat is not available (perhaps an X)
    
    #is seat available?
    function seat_availability(seat_number):
      seat number contains row and column
        if seat is avilable:
          return true
        else:
          return false
    
    #assigning ticket ID/reference
    function ticket_ID(seat_number, user_id, event_code)
      based on event code, user_id and seat_number, generate a ticket ID (could be random)
      show ticket id
    
    #booking the seat (turning it into unavailable)
    function seat_booked(seat_number)
      if seat_availability(seat_number) is true
        mark seat as not available

    #booking the seat
    function booking_seats(seat_number)
      if seat_availability(seat_number) is true
        seat_booked(seat_number)
        ticket_ID(seat_number, user_id, event_code)
    
    #if event is full
    function waitlist_add()
      for each seat in grid
        if seat is available
          return False (it is still bookable)
      return true (not bookable, waitlist add)
      if True
        use some timastamp for priority in booking
        add user ID and contact info to list (or perhaps dict)


    #adding user to waitlist
    #viewing the waitlist
    
         



In [1]:
import uuid  #to give unique user and ticket id
import string  #to get ascii uppercase to be uses in seat codes

class TicketingSystem:
  def __init__(self, event_code, rows=20, columns=25):
    """
    initialises a ticketing system with a grit with seats, ticket info (empty) and waitlist
    """
    self.event_code = event_code    #code for the event (mix of letters and numbers)
    self.rows = rows                #number of rows with seats, default 20
    self.columns = columns         #number of columns with seats, default 25
    self.grid = self.seat_grid_create()         #empty list to represent the grid
    self.ticket_info = {}  #dictionary to store information about the ticket
    self.waitlist = WaitlistQueue()     #connecting to class storing waitlist

  #seat grid method
  def seat_grid_create(self):
    """
    starts an empty seat grid with rows and columns defined by the class by default
    :param: rows and columns (both optional)
    :return: grid
    """
    grid = []                     #starting empty list (represent rows)
    for _ in range(self.rows):   #going through each row
      row = []                   #empty row
      for _ in range(self.columns):  #for each row and each column
        row.append(None)             #add none
      grid.append(row)               #adding another row to the grid
    return grid                  #return the entire grid

  #showing grid of seats
  def show_seats(self):
    """
    displays the available seats in a visual way
    :param: none
    :print: available and booked visual info
    """
    for r in range(self.rows):                #for each row
      row_letter = string.ascii_uppercase[r]  #to have rows starting from A to T
      for c in range (self.columns):          #for each column in the row
        seat_code = f"{row_letter}{c + 1}"    #row letter and column number (1 to 25) should form the seat code
        print(f" {seat_code:02}", end = "")   #printing the code for the seat
        if self.grid[r][c] is None:           #checking if seat is available
          print("🔲", end=" ")               #showing with the empty square symbol that seat is available
        else:
          print("⬛", end=" ")               #showing with the filled square symbol that seat is not available
      print()                               #breking the line to show grid properly
    print(f"🔲 = Available seat \n⬛ = Booked seat")  #showing meaning of squares to my user


  #turning seat number into position in grid
  def seat_code_position(self, seat_code):
    """
    locates where seat code (A1, A2, B12, C22, ...,) is located in the grid
    :param: seat_code (string being A1, A2, B12, C23...)
    :return: position with row and column of seat
    """
    row_letter = seat_code[0].upper()             #getting first char as the letter and make it uppercase
    column_number = int(seat_code[1:])            #converting everything after the letter to an integer to get column

    row = string.ascii_uppercase.index(row_letter)  #getting index of letter in alphabet
    column = column_number - 1                      #removing 1 from column number to be aligned with index

    #printing information
    return row, column

  #checking seat availability
  def seat_availability(self, seat_code):
    """
    checks if seat is available or not and prints information to user
    :param: seat_number
    :return: true if seat is available, false if seat is not available
    """
    row, column = self.seat_code_position(seat_code)  #calling method to locate seat in grid
    if self.grid[row][column] is None:                #if grid cell is available
      #print(f"{seat_code} is available.")             it was causing duplication when calling this method in another method
      return True
    else:
      #print(f"{seat_code} is not available. Select another one.")
      return False

  #generating ticket id
  def generate_ticket_id(self, seat_code, user_id):
    """
    generates a ticket id using event code, user id and seat code as references
    """
    return f"{self.event_code}-{user_id[:4]}-{seat_code}"  #example: EVENTNAME-ab12-D12

  #marking seat as booked
  def from_available_to_booked(self, seat_code, ticket_id):
    """
    assumes seat is available and marks it as booked and saves the ticket ID in the grid
    """
    row, column = self.seat_code_position(seat_code)  #calling method to locate seat in grid
    self.grid[row][column] = ticket_id                #attaching seat position to the ticket id
    return True

  def booking_seat(self, seat_code, name, email, priority=3):
    """
    checks seat availability, generates ID, marks seat as booked, stores ticket info
    """
    if self.seat_availability(seat_code) == False:    #checking if seat is already booked first
      print(f"The seat {seat_code} is not available. Select a different seat.")

    else:
      user_id = str(uuid.uuid4())                                       #generating user id
      ticket_id = self.generate_ticket_id(seat_code, user_id)           #generating a ticket id

      self.from_available_to_booked(seat_code, ticket_id)       #marking seat as booked

      #storing ticket info to dictionary with user info as well
      self.ticket_info[ticket_id] = {"name": name,
                                     "email": email,
                                     "seat": seat_code,
                                     "ticket_id": ticket_id,
                                     "user_id": user_id,
                                     "priority": priority}

      print(f"Seat {seat_code} booked! Ticket ID is {ticket_id}")   #print message to user with ticket


  def is_event_full(self):
    """
    checks if all seats are booked
    :param: none
    :return: True if all seats are booked and false if there are still seats available
    """
    for row in self.grid:      #going through each row
      for seat in row:         #going through each seat in row
        if seat is None:       #if seat is still available
          return False         #returning false
    return True                #if all seats are booked, return true

  def add_to_waitlist(self, name, email):
    """
    adds user to waitlist using class for waitlist
    """
    if self.is_event_full():               #checking if event is full with method
      user_id = str(uuid.uuid4())                  #generating user id
      self.waitlist.add_user(name, email, user_id)       #using method in waitlist class to add users to waitlist

      print(f"{name}, you were added to the waitlist.")
    else:
      print("There are still available seats. Book now!")    #if seats are still available, user cannot be added to waitlist


  def display_waitlist(self):
    """
    shows the waitlist ordered by time of request based on waitlist class method
    """
    self.waitlist.display_waitlist()

  #need to test waitlist if all seats are booked, so adding this function here
  def book_all_seats(self):
    """
    books all seats in the grid with placeholder info to simulate it's fully booked to test if waitlist is working properly
    """
    for r in range(self.rows):         #for each row
        for c in range(self.columns):  #for each seat in row
            seat_code = f"{string.ascii_uppercase[r]}{c + 1}"        #creating seat code based on the row letter and the column number
            user_id = str(uuid.uuid4())                              #getting a random id for user
            ticket_id = self.generate_ticket_id(seat_code, user_id)  #getting random ticket id
            priority = 3                                             #setting priority default to 3

            self.grid[r][c] = ticket_id                              #assigning ticket to seat

            self.ticket_info[ticket_id] = {                          #adding placeholder information to dictionary that stores info
                "name": f"test{r}{c}",
                "email": f"test{r}{c}@email.com",
                "seat": seat_code,
                "ticket_id": ticket_id,
                "user_id": user_id,
                "priority": priority}
    print("all seats booked with placeholders")



In [2]:
def user_priority(user):   #function to sort by priority
  return user["priority"]  #the lower the number, the higher the priority

class WaitlistQueue:
  """
  priority queue based on time of addition
  earliest added user is the first one to be assigned a seat if any becomes available
  """
  def __init__(self):
    self.queue = []   #empty list to represent waitlist

  def add_user(self, name, email, user_id, priority=3):
    """
    adds a new user to the end of the list (queue)
    priority is 3 by default, common users
    special users would have priority 2 and VIPs would have priority 1
    """
    #adding user information as a dictionary
    self.queue.append({"name": name, "email": email, "user_id": user_id, "priority": priority})

  def next_waitlisted_person(self):   #acting as dequeue
    """
    removes the first person in the list (number 1 priority)
    :return: user info (dictionary) or none if list is empty
    """
    if self.is_waitlist_empty():   #if queue is empty
      return None

    self.queue.sort(key = user_priority) #soting by priority and keeping original order in case of same priority numbers
    return self.queue.pop(0)   #removing ans returning the first person in line

  def is_waitlist_empty(self):
    """
    checks if waitlist is empty
    """
    if len(self.queue) == 0:
      print("Waitlist is empty.")
      return True
    else:
      print("Waitlist:")
      return False

  def display_waitlist(self):
    """
    prints waitlist in order of priority
    """
    if not self.is_waitlist_empty():
      for priority, user in enumerate(self.queue, start=1):
        print(f"{priority}. {user['name']}, ID: {user['user_id']}")

In [3]:
"""
These are the references I used for this part of the assignment
{seat_number:03} defines the number of charactes to be shown
https://docs.python.org/3/tutorial/inputoutput.html

row = string.ascii_uppercase.index(row_letter) gets the uppercase value of letters from A to Z
https://docs.python.org/3/library/string.html#string.ascii_uppercase

abc

"""

'\nThese are the references I used for this part of the assignment\n{seat_number:03} defines the number of charactes to be shown\nhttps://docs.python.org/3/tutorial/inputoutput.html\n\nrow = string.ascii_uppercase.index(row_letter) gets the uppercase value of letters from A to Z\nhttps://docs.python.org/3/library/string.html#string.ascii_uppercase\n\nabc\n\n'

# B. Ticket Cancellation:

* The program will allow customers to cancel a booked ticket using their Ticket ID/Reference.
* After cancellation, the program will update the dynamic array to reflect the available
seat.
* Automatically assign the seat to the next customer on the waitlist (if any).


In [4]:
def cancel_booking(self, ticket_id):
  """
  allows user to cancel a booked seat, if waitlist exists, get priority person and assigns to the seat
  :param: ticket_id
  :return: updates seat from booked to available if waitlist is empty, otherwise, keeps it as booked and moves user from waitlist to seat
  """
  if ticket_id not in self.ticket_info:  #if ticket_id not in system, return
    print("Ticket not found.")
    return

  seat_code = self.ticket_info[ticket_id]["seat"]   #getting seat code from ticket info
  row, column = self.seat_code_position(seat_code)  #converting seat code to where it's located

  del self.ticket_info[ticket_id]   #removing booking

  new_user = self.waitlist.next_waitlisted_person()   #checking if we have priority 1 in waitlits

  if new_user:    #if we have someone in waitlist
    self.booking_seat(seat_code, new_user["name"], new_user["email"])  #booking seat to priority 1 waitlisted person
    print(f"{new_user['name']} is now booked for seat {seat_code}.")

  else:     #if waitlist is empty
    self.grid[row][column] = None    #making seat empty (available for booking)
    print(f"Ticket {ticket_id} is cancelled.")

In [5]:
#attaching cancel_booking to ticketingsystem class
TicketingSystem.cancel_booking = cancel_booking

# C. Modify Ticket:

* The program will allow customers to modify their existing booking (e.g. change seat number or priority).
* After any modification, it will update all relevant data structures to reflect the change.

In [27]:
def modify_ticket(self, ticket_id, new_seat_code= None, new_priority= None):
  """
  allows customer to modify their existing booked seat or their priority level
  updates structures based on changes
  """
  #checking if ticket exists in system
  if ticket_id not in self.ticket_info:
    print("Ticket not found.")
    return

  #getting booking details already registered in the system
  current_info = self.ticket_info[ticket_id]  #getting booking information stored in the system
  current_seat = current_info["seat"]         #getting current seat code
  user_id = current_info["user_id"]           #getting user ID from ticket info
  current_priority = self.ticket_info[ticket_id]["priority"]

  #checking if change in seat
  if new_seat_code and new_seat_code != current_seat:   #if new seat exists and is different from the current one
    if self.seat_availability(new_seat_code):                 #checking if the new seat is free
      row_old, column_old = self.seat_code_position(current_seat)  #finding position of old seat in grid
      self.grid[row_old][column_old] = None                       #making previous seat available again (none)

      #booking new seat
      row_new, column_new = self.seat_code_position(new_seat_code)  #getting position of new seat in grid
      self.grid[row_new][column_new] = ticket_id               #assigning ticket id to new seat

      #updating seat information in ticket details
      self.ticket_info[ticket_id]["seat"] = new_seat_code  #storing new seat code

      print(f"Seat updated from {current_seat} to {new_seat_code}.")
    else:
      print(f"Seat {new_seat_code} is not available.")  #if seat is already occupied

  #if priority is changed
  if new_priority != current_priority:
    self.ticket_info[ticket_id]["priority"] = new_priority
    print(f"User priority updated to {new_priority}")
  else:
    new_priority = current_priority



In [28]:
#attaching modify_ticket to the ticketingsystem class
TicketingSystem.modify_ticket = modify_ticket

# D. Priority Handling and Dynamic Array Expansion:

* Multiple customers are on waiting-list with priority and the concert gains even more popularity. Priority levels determine the order in which seats are reassigned to a customer due to cancellation by other customer.

 * For example: A customer named as Grace with priority 1, another customer:
Henry with priority 2, and finally Irene with priority 3 are on the waitlist.
 * Let’s assume that Seat 20 becomes available (a customer cancels the booking),
then the system assigns it to Grace first, since she has the highest priority.

* Now the concert organizer decides to increase the seating capacity to 600 due to overwhelming demand. The system should dynamically expand the seat array to
accommodate the additional 100 seats, enabling new bookings.

In [29]:
#priority was already handled in the cells before

def increase_seating_capacity(self, extra_seats):
  """
  increases the seating capacity by adding new rows
  does not change column structure
  """
  current_total = self.rows * self.columns        #checking total number of seats before expansion
  new_total = current_total + extra_seats         #assigning total numbe of seats after increasing
  new_rows = (new_total + self.columns - 1) // self.columns  #calculating how many rows are needed for the new number

  if new_rows > len(string.ascii_uppercase):          #avoiding issues with increasing it for beyond the letter z
    print("Cannot expand beyond 26 rows (A to Z).")
    return

  for row in range(self.rows, new_rows):        #adding rows starting from the last row at the moment
    self.grid.append([None] * self.columns)

  self.rows = new_rows           #updating number of rows
  print(f"Grid was increased. New seats should be available.")


In [30]:
#attaching increase_seating capacity to the ticketingsystem class
TicketingSystem.increase_seating_capacity = increase_seating_capacity

In [10]:
#system.expand_seating_capacity(100)

# E. Hash Table for Fast Lookups:

A customer calls to inquire about their booking. The system quickly retrieves the ticket
details for Ticket ID: 150 using the hash table:
* Ticket ID: 150AT2
* Name: Andrew Thomas
* Seat: 150
* Priority: 2

In [31]:
#I adapted the requested information to the way my code was developed (in terms of style of ticket id, name and seat code)
def check_ticket_details(self, ticket_id):
  """
  retrieves and shows booking details based on ticket id
  """
  if ticket_id in self.ticket_info:  #checking if ticket exists
    details = self.ticket_info[ticket_id]  #assigning details variable for ticket id
    print(f"Ticket ID: {ticket_id}")
    print(f"Name: {details['name']}")
    print(f"Seat: {details['seat']}")

  else:
    print(f"There is not ticket_id {ticket_id} registered")


In [32]:
#attaching check ticket details to the ticketing system class
TicketingSystem.check_ticket_details = check_ticket_details

In [33]:
system = TicketingSystem("EDM25")  #creating instance of the ticketing system for the event

Using a user friendly menu to be able to test all methods:

In [34]:
def run_ticketing_system():

  while True:                             #loop to run until manually stopped
    print("\n TicketingSystem Menu. WHat would you like to do?")     #title to present options
    print("1. View seating map")
    print("2. Book a seat")
    print("3. Join the waitlist")
    print("4. See the waitlist")
    print("5. Check if all tickets are sold")
    print("6. (Testing) Booking all seats")   #this is just for testing if waitlist is working and visual grid
    print("7. Make a cancellation")
    print("8. Modify reservation")
    print("9. Increase seating capacity")
    print("10. Check ticket details")
    print("11. Exit")

    choice = input("Select an option (1 to 11): ")  #asking user for input 1 to 11

    if choice == "1":
      system.show_seats()    #calling method to show seat grid

    elif choice == "2":
      try:
        seat_row_letter = input("Enter the letter corresponding to your seat row (A to T): ")   #asking user for desired row
        seat_col_number = input("Enter the number corresponding to your seat column (1–25): ")  #asking user for desired seat position in row
        seat_number = seat_row_letter + seat_col_number    #combining the input to the seat code

        if system.seat_availability(seat_number):       #checking if seat selected by user is available
          name = input("Enter your name: ")             #asking for name
          email = input("Enter your email: ")           #asking for email
          try:
            priority = int(input("Enter your priority level (1 = VIP, 2 = Special, 3 = Regular): "))
            if priority not in [1, 2, 3]:
              priority = 3
          except ValueError:
            priority = 3

          system.booking_seat(seat_number, name, email)  #calling method to book
        else:
          print(f"Seat {seat_number} is not available. Please choose another.")
      except ValueError:
        print("Please enter a seat number between 1 and 500.")

    elif choice == "3":
      name = input("Enter your name: ")   #asking for name to add to waitlist
      email = input("Enter your email: ")  #asking for email
      system.add_to_waitlist(name, email)  #calling function to add to waitlist

    elif choice == "4":
      system.display_waitlist()    #showing waitlist

    elif choice == "5":
      if system.is_event_full():    #checking if event is full
        print("There are no available seats.")
      else:
        print("There are still seats available.")

    elif choice == "6":
      system.book_all_seats()    #using test method to book all seats

    elif choice == "7":
      ticket_id = input("Enter your Ticket ID to cancel: ")
      system.cancel_booking(ticket_id)    #using cancel booking method

    elif choice == "8":
      ticket_id = input("Enter your Ticket id: ")
      new_seat = None   #default is none
      new_priority = None   #default is none

      #seat change
      change_seat = input("Would you like to change your seat? (Yes or No) ").upper()   #turning user input in uppercase
      if change_seat == "YES":
        new_seat_row_letter = input("Enter the letter corresponding to your new seat row (A to T): ")   #asking user for desired row
        new_seat_col_number = input("Enter the number corresponding to your new seat column (1–25): ")  #asking user for desired seat position in row
        new_seat = new_seat_row_letter + new_seat_col_number    #combining the input to the seat code

      #priority change
      change_priority = input("Would you like to change the priority? (Tes or No) ").upper()
      if change_priority == "YES":
        try:
          new_priority = int(input("Enter new priority (1, 2 or 3): "))
          if new_priority not in [1, 2, 3]:
            new_priority = 3
        except ValueError:
          new_priority = 3

      #calling method to run the changes
      system.modify_ticket(ticket_id, new_seat_code=new_seat, new_priority=new_priority)  #calling method defined earlier, assigning new seat code from method to varible created here "new_seat"

    elif choice == "9":
      extra_seats = int(input("How many extra seats to add? "))  #asking for number of seats to add
      system.increase_seating_capacity(extra_seats)  #calling method to increase seating capacity

    elif choice == "10":
      ticket_id = input("Enter your Ticket id: ")
      system.check_ticket_details(ticket_id)

    elif choice == "11":
      print("Bye bye!")
      break

    else:
      print("Invalid choice. Please try again.")

In [35]:
run_ticketing_system()


 TicketingSystem Menu. WHat would you like to do?
1. View seating map
2. Book a seat
3. Join the waitlist
4. See the waitlist
5. Check if all tickets are sold
6. (Testing) Booking all seats
7. Make a cancellation
8. Modify reservation
9. Increase seating capacity
10. Check ticket details
11. Exit
Select an option (1 to 11): 1
 A1🔲  A2🔲  A3🔲  A4🔲  A5🔲  A6🔲  A7🔲  A8🔲  A9🔲  A10🔲  A11🔲  A12🔲  A13🔲  A14🔲  A15🔲  A16🔲  A17🔲  A18🔲  A19🔲  A20🔲  A21🔲  A22🔲  A23🔲  A24🔲  A25🔲 
 B1🔲  B2🔲  B3🔲  B4🔲  B5🔲  B6🔲  B7🔲  B8🔲  B9🔲  B10🔲  B11🔲  B12🔲  B13🔲  B14🔲  B15🔲  B16🔲  B17🔲  B18🔲  B19🔲  B20🔲  B21🔲  B22🔲  B23🔲  B24🔲  B25🔲 
 C1🔲  C2🔲  C3🔲  C4🔲  C5🔲  C6🔲  C7🔲  C8🔲  C9🔲  C10🔲  C11🔲  C12🔲  C13🔲  C14🔲  C15🔲  C16🔲  C17🔲  C18🔲  C19🔲  C20🔲  C21🔲  C22🔲  C23🔲  C24🔲  C25🔲 
 D1🔲  D2🔲  D3🔲  D4🔲  D5🔲  D6🔲  D7🔲  D8🔲  D9🔲  D10🔲  D11🔲  D12🔲  D13🔲  D14🔲  D15🔲  D16🔲  D17🔲  D18🔲  D19🔲  D20🔲  D21🔲  D22🔲  D23🔲  D24🔲  D25🔲 
 E1🔲  E2🔲  E3🔲  E4🔲  E5🔲  E6🔲  E7🔲  E8🔲  E9🔲  E10🔲  E11🔲  E12🔲  E13🔲  E14🔲  E15🔲  E16🔲  E17🔲  E18🔲  E19

KeyboardInterrupt: Interrupted by user

# F. Use of GenAI for Improvment

In this part you are required to the use of Generative AI to optimize and enhance the Dynamic Ticketing System. By leveraging AI tools, you will:
* Brainstorm ideas for improving system functionality and efficiency such as implementing group booking, student discount, senior discount, refund tickets etc. You can select any one or two features or if you can think of a new feature, add that too.

 * Generate test data for edge cases and performance testing such as multiple simultaneous booking for the same seat.
 * Analyze and suggest improvements to their existing code such as reduce time complexity for booking, cancellation and modify, optimizing data structures for memory efficiency.
 * Documentation and Reflection: Summarize the insights gained from using Generative AI in this part. You MUST provide chat logs, screen shots of using GenAI.
 * Reflect on how AI tools contributed to the overall development and optimization of the
ticketing system

In [36]:
import uuid  #to give unique user and ticket id
import string  #to get ascii uppercase to be uses in seat codes

class TicketingSystem:
  def __init__(self, event_code, rows=20, columns=25):
    self.event_code = event_code
    self.rows = rows
    self.columns = columns
    self.grid = self.seat_grid_create()
    self.ticket_info = {}
    self.user_registry = {}  # user_id: list of ticket_ids
    self.waitlist = WaitlistQueue()

  def seat_grid_create(self):
    grid = []
    for _ in range(self.rows):
      row = []
      for _ in range(self.columns):
        row.append(None)
      grid.append(row)
    return grid

  def show_seats(self):
    for r in range(self.rows):
      row_letter = string.ascii_uppercase[r]
      for c in range(self.columns):
        seat_code = f"{row_letter}{c + 1}"
        print(f" {seat_code:02}", end="")
        print("🔲" if self.grid[r][c] is None else "⬛", end=" ")
      print()
    print(f"🔲 = Available seat \n⬛ = Booked seat")

  def seat_code_position(self, seat_code):
    row_letter = seat_code[0].upper()
    column_number = int(seat_code[1:])
    row = string.ascii_uppercase.index(row_letter)
    column = column_number - 1
    return row, column

  def seat_availability(self, seat_code):
    row, column = self.seat_code_position(seat_code)
    return self.grid[row][column] is None

  def generate_ticket_id(self, seat_code, user_id):
    return f"{self.event_code}-{user_id[:4]}-{seat_code}"

  def from_available_to_booked(self, seat_code, ticket_id):
    row, column = self.seat_code_position(seat_code)
    self.grid[row][column] = ticket_id
    return True

  def booking_seat(self, seat_code, name, email, user_id=None, priority=3):
    if not self.seat_availability(seat_code):
      print(f"The seat {seat_code} is not available. Select a different seat.")
      return False

    if user_id is None:
      user_id = str(uuid.uuid4())
      print(f"New user created. Your User ID is: {user_id}")

    ticket_id = self.generate_ticket_id(seat_code, user_id)
    self.from_available_to_booked(seat_code, ticket_id)

    self.ticket_info[ticket_id] = {
      "name": name,
      "email": email,
      "seat": seat_code,
      "ticket_id": ticket_id,
      "user_id": user_id,
      "priority": priority
    }

    if user_id not in self.user_registry:
      self.user_registry[user_id] = []
    self.user_registry[user_id].append(ticket_id)

    print(f"Seat {seat_code} booked! Ticket ID is {ticket_id}")
    return True

  def suggest_adjacent_seats(self, num_tickets):
    for r in range(self.rows):
      for c in range(self.columns - num_tickets + 1):
        seats = [f"{string.ascii_uppercase[r]}{c + i + 1}" for i in range(num_tickets)]
        if all(self.seat_availability(seat) for seat in seats):
          return seats
    return []

  def group_booking(self, name, email, num_tickets, user_id=None, priority=3):
    if user_id is None:
      user_id = str(uuid.uuid4())
      print(f"New user created. Your User ID is: {user_id}")
    elif user_id not in self.user_registry:
      self.user_registry[user_id] = []

    if len(self.user_registry.get(user_id, [])) + num_tickets > 5:
      print("You cannot book more than 5 tickets per user.")
      return

    if num_tickets > 1:
      suggested_seats = self.suggest_adjacent_seats(num_tickets)
      if suggested_seats:
        print("Suggested adjacent seats:", suggested_seats)
        confirm = input("Do you want to book these? (Yes/No): ").strip().upper()
        if confirm == "YES":
          for seat in suggested_seats:
            self.booking_seat(seat, name, email, user_id, priority)
          return

    print("Please enter your desired seat codes one by one:")
    selected_seats = []
    while len(selected_seats) < num_tickets:
      seat = input(f"Enter seat code {len(selected_seats)+1} of {num_tickets}: ").strip().upper()
      if self.seat_availability(seat):
        selected_seats.append(seat)
      else:
        print(f"Seat {seat} is not available. Choose another.")

    for seat in selected_seats:
      self.booking_seat(seat, name, email, user_id, priority)

  def is_event_full(self):
    for row in self.grid:
      for seat in row:
        if seat is None:
          return False
    return True

  def add_to_waitlist(self, name, email):
    if self.is_event_full():
      user_id = str(uuid.uuid4())
      self.waitlist.add_user(name, email, user_id)
      print(f"{name}, you were added to the waitlist.")
    else:
      print("There are still available seats. Book now!")

  def display_waitlist(self):
    self.waitlist.display_waitlist()

  def book_all_seats(self):
    for r in range(self.rows):
      for c in range(self.columns):
        seat_code = f"{string.ascii_uppercase[r]}{c + 1}"
        user_id = str(uuid.uuid4())
        ticket_id = self.generate_ticket_id(seat_code, user_id)
        priority = 3
        self.grid[r][c] = ticket_id
        self.ticket_info[ticket_id] = {
          "name": f"test{r}{c}",
          "email": f"test{r}{c}@email.com",
          "seat": seat_code,
          "ticket_id": ticket_id,
          "user_id": user_id,
          "priority": priority
        }
    print("all seats booked with placeholders")


In [37]:
class WaitlistQueue:
  def __init__(self):
    self.queue = []

  def add_user(self, name, email, user_id, priority=3):
    self.queue.append({"name": name, "email": email, "user_id": user_id, "priority": priority})

  def next_waitlisted_person(self):
    if self.is_waitlist_empty():
      return None
    self.queue.sort(key=lambda user: user["priority"])
    return self.queue.pop(0)

  def is_waitlist_empty(self):
    return len(self.queue) == 0

  def display_waitlist(self):
    if not self.is_waitlist_empty():
      for idx, user in enumerate(self.queue, start=1):
        print(f"{idx}. {user['name']}, ID: {user['user_id']}")
    else:
      print("Waitlist is empty.")


In [38]:
import unittest

class TestTicketingSystem(unittest.TestCase):
    def setUp(self):
        self.system = TicketingSystem("TEST", rows=5, columns=5)

    def test_suggest_adjacent_seats(self):
        seats = self.system.suggest_adjacent_seats(3)
        self.assertEqual(len(seats), 3)
        self.assertTrue(all(self.system.seat_availability(seat) for seat in seats))

    def test_group_booking_success(self):
        user_id = self.system.group_booking("Alice", "alice@example.com", 2)
        self.assertIsInstance(user_id, str)  # Because user_id is printed and returned
        self.assertEqual(len(self.system.user_registry[user_id]), 2)

    def test_group_booking_respects_limit(self):
        user_id = "test-user"
        self.system.user_registry[user_id] = []
        self.system.group_booking("Alice", "alice@example.com", 3, user_id=user_id)
        self.system.group_booking("Alice", "alice@example.com", 2, user_id=user_id)
        result = self.system.group_booking("Alice", "alice@example.com", 1, user_id=user_id)
        self.assertEqual(result, None)  # Because print() is used, not return
        self.assertEqual(len(self.system.user_registry[user_id]), 5)

# Run the test suite in Colab
unittest.main(argv=[''], verbosity=2, exit=False)


test_group_booking_respects_limit (__main__.TestTicketingSystem.test_group_booking_respects_limit) ... 

Suggested adjacent seats: ['A1', 'A2', 'A3']
Do you want to book these? (Yes/No): yes
Seat A1 booked! Ticket ID is TEST-test-A1
Seat A2 booked! Ticket ID is TEST-test-A2
Seat A3 booked! Ticket ID is TEST-test-A3
Suggested adjacent seats: ['A4', 'A5']


KeyboardInterrupt: Interrupted by user